In [1]:
import os
import pandas as pd
import numpy as np
import sys
import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression, LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
import sklearn.metrics as metrics
from sklearn.metrics import brier_score_loss, roc_curve, auc, confusion_matrix, roc_auc_score, mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from plotnine import *
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from stargazer import stargazer
from statsmodels.tools.eval_measures import mse,rmse
import matplotlib.pyplot as plt
from mizani.formatters import percent_format

import warnings
warnings.filterwarnings('ignore')

data = pd.read_csv("https://raw.githubusercontent.com/Jk33033/DA3_hw3/main/cs_bisnode_panel.csv")


In [2]:
data.describe()

,comp_id,COGS,amort,curr_assets,curr_liab,extra_exp,extra_inc,extra_profit_loss,finished_prod,fixed_assets,...,ceo_count,foreign,female,birth_year,inoffice_days,nace_main,ind2,ind,urban_m,labor_avg
count,2.878290e+05,1.825700e+04,2.797890e+05,2.876980e+05,2.876980e+05,2.693000e+05,2.693000e+05,2.706260e+05,1.748500e+04,2.876980e+05,...,231402.000000,231402.000000,231402.000000,176011.000000,231402.000000,286791.000000,286791.000000,278060.000000,287829.000000,141297.000000
mean,1.540862e+11,4.601683e+05,2.353264e+04,2.209395e+05,1.742499e+05,1.521749e+03,3.754458e+03,2.229149e+03,1.056481e+05,3.393731e+05,...,1.254038,0.115653,0.255157,1964.902661,3205.323629,4858.231479,48.353062,2.692023,2.067075,1.009435
std,1.382424e+11,2.614149e+06,8.216668e+05,4.182020e+06,3.730357e+06,7.237342e+04,1.727794e+05,1.611169e+05,5.338053e+05,1.823176e+07,...,0.517837,0.308518,0.406538,11.546875,1880.553038,1250.012437,12.570550,0.512269,0.847473,4.771655
min,1.001034e+06,-8.445704e+05,-3.076704e+05,-5.555556e+05,-7.727111e+05,-4.902703e+05,-2.744074e+05,-6.181263e+06,0.000000e+00,-2.394226e+06,...,1.000000,0.000000,0.000000,1920.000000,1.000000,111.000000,1.000000,1.000000,1.000000,0.000000
25%,2.868635e+10,0.000000e+00,3.333333e+01,2.203704e+03,1.833333e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.000000,0.000000,0.000000,1956.000000,1775.000000,3314.000000,33.000000,2.000000,1.000000,0.090909
50%,1.152850e+11,7.040741e+03,7.814815e+02,9.592593e+03,1.049259e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.825926e+03,...,1.000000,0.000000,0.000000,1966.000000,2838.000000,5610.000000,56.000000,3.000000,2.000000,0.243056
75%,2.593958e+11,1.063000e+05,4.003704e+03,3.552130e+04,4.101852e+04,0.000000e+00,0.000000e+00,0.000000e+00,3.592592e+02,3.593982e+04,...,1.000000,0.000000,0.500000,1974.000000,4468.750000,5610.000000,56.000000,3.000000,3.000000,0.583333
max,4.641209e+11,9.499693e+07,1.820889e+08,1.299011e+09,1.683374e+09,1.704172e+07,7.233078e+07,7.233078e+07,1.803635e+07,7.940211e+09,...,15.000000,1.000000,1.000000,2016.000000,10983.000000,9609.000000,96.000000,3.000000,3.000000,219.333344


In [5]:
data.columns

Index(['comp_id', 'begin', 'end', 'COGS', 'amort', 'curr_assets', 'curr_liab',
       'extra_exp', 'extra_inc', 'extra_profit_loss', 'finished_prod',
       'fixed_assets', 'inc_bef_tax', 'intang_assets', 'inventories',
       'liq_assets', 'material_exp', 'net_dom_sales', 'net_exp_sales',
       'personnel_exp', 'profit_loss_year', 'sales', 'share_eq',
       'subscribed_cap', 'tang_assets', 'wages', 'D', 'balsheet_flag',
       'balsheet_length', 'balsheet_notfullyear', 'year', 'founded_year',
       'exit_year', 'ceo_count', 'foreign', 'female', 'birth_year',
       'inoffice_days', 'gender', 'origin', 'nace_main', 'ind2', 'ind',
       'urban_m', 'region_m', 'founded_date', 'exit_date', 'labor_avg'],
      dtype='object')

# Hold-out sample

In [13]:
data["comp_id"].describe

<bound method NDFrame.describe of 0         1.001034e+06
1         1.001034e+06
2         1.001034e+06
3         1.001034e+06
4         1.001034e+06
              ...     
287824    4.641209e+11
287825    4.641209e+11
287826    4.641209e+11
287827    4.641209e+11
287828    4.641209e+11
Name: comp_id, Length: 287829, dtype: float64>

" We are only interested in predicting default for ‘ind2 == 26’, which is the selected industry, and the
firm is a small or medium enterprise (SME). Thus, yearly sales in 2014 was between 1000 EUR and 10
million EUR."

In [69]:
# Hold-out sample
data_holdout = data[data['ind2'] == 26] 
data_holdout_id = data_holdout[(data_holdout['year'] == 2014) & 
                         (data_holdout['sales'] >= 1000) & 
                         (data_holdout['sales'] <= 10000000)]['comp_id'].unique()# group by id
data_holdout = data_holdout[data_holdout['comp_id'].isin(data_holdout_id)]#data_holdout has all years of the id 

" If you do the sample design properly, you have an overall of 1037 firms. 56 firms defaulted, and 981
stayed alive. The average sales of the firms is 0.4902 million EUR, with the minimum of 0.00107 million
EUR and 9.57648 million EUR."

In [70]:
# check the num of the company
len(data_holdout_id)

1037

The definition of default should be the following:
– Existed in 2014 (sales > 0), but did not exists in 2015 (sales is 0 or missing)

In [71]:
#check if there are 981 firms that stayed alive
#already filtered for existence in 2014 above
data_holdout_default_id = data_holdout[(data_holdout['year'] == 2015) 
                                    & (data_holdout['sales'] > 0 )]['comp_id'].unique()
len(data_holdout_default_id)

981